 ## Global Imports And Library Setup

In [1]:
# # Uncomment below to install missing packages
# !python -m pip install --upgrade --requirement ./requirements.txt


 ### Login to Azure CLI

In [2]:
# # Login to Azure CLI
# !az login --use-device-code --tenant "38b2262e-92fe-4b71-a4d0-ebf91a3e2909"
# !az account set --subscription "47c2af6c-fe2f-4dbd-9193-9b50a99044b7"


 ### Import Python libraries

In [1]:
import os
import re
import ast
import json
import time
import uuid
import openai
import logging
import requests
import pandas as pd
from PIL import Image
from io import BytesIO
from pathlib import Path
from tqdm.auto import tqdm
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Load environment variables from `.env` file
load_dotenv(verbose=True, override=True)


True

 ### Setup environment constants

In [4]:
# Azure container name
CONTAINER_NAME = os.environ.get("CONTAINER_NAME", "content-creation-pipeline")

try:
    SYNC_BLOB_TO_LOCAL_FORCE_DOWNLOAD = ast.literal_eval(
        os.environ.get("SYNC_BLOB_TO_LOCAL_FORCE_DOWNLOAD", "False")
    )
except:
    SYNC_BLOB_TO_LOCAL_FORCE_DOWNLOAD = False


 ## Clean Azure Data (If Needed)

In [5]:
# # Used to clean up the blob if doing a bulk-delete
# ################################################################################################
# container_client = blob_service_client.get_container_client(container=CONTAINER_NAME)
# for n in container_client.list_blob_names():
#     if not n.startswith("data"):
#         continue
#     folderSplits = n.split("/")
#     if len(folderSplits) != 3:
#         continue
#     try:
#         puzzleId = int(n.split("/")[1], base=10)
#         if puzzleId > 12:
#             print(f"Deleting blob = {n}")
#             blob_service_client.get_blob_client(container=container_name, blob=n).delete_blob()
#     except ValueError as ex:
#         if ex.args[0].startswith("invalid literal for int() with base 10"):
#             # This is an ad-hoc file. And not a puzzle folder
#             continue
#         else:
#             print(ex)
#             break


 ## Setup Azure Libraries

In [6]:
# account_url = "https://rakhdelstudioapps.blob.core.windows.net"
# default_credential = DefaultAzureCredential(
#     exclude_environment_credential=True,
#     exclude_managed_identity_credential=True,
#     exclude_visual_studio_code_credential=True,
#     exclude_shared_token_cache_credential=True,
#     interactive_browser_tenant_id="38b2262e-92fe-4b71-a4d0-ebf91a3e2909",
# )

# # Create the BlobServiceClient object
# blob_service_client = BlobServiceClient(account_url, credential=default_credential)


In [9]:
import azure.cognitiveservices.speech as speechsdk
import wave

# This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
speech_config = speechsdk.SpeechConfig(
    subscription=os.environ.get("SPEECH_KEY"),
    region=os.environ.get("SPEECH_REGION"),
    speech_recognition_language="gu-IN-DhwaniNeural",
)

language = "gu-IN"
speech_config.speech_synthesis_language = language
# The language of the voice that speaks.
speech_config.speech_synthesis_voice_name = "gu-IN-DhwaniNeural"


def generate_transcription(text: str, output_wave_file: str) -> None:
    audio_config = speechsdk.audio.AudioOutputConfig(
        filename=output_wave_file,
    )

    speech_synthesizer = speechsdk.SpeechSynthesizer(
        speech_config=speech_config, audio_config=audio_config
    )

    ssml = (
        """
    <!--ID=B7267351-473F-409D-9765-754A8EBCDE05;Version=1|{
        "VoiceNameToIdMapItems":[
            {
                "Name": "Microsoft Server Speech Text to Speech Voice (gu-IN, DhwaniNeural)",
                "ShortName": "gu-IN-DhwaniNeural",
                "Locale": "gu-IN",
                "Id": "97ebc7c6-1e92-4764-806b-ad61201a60a5",
                "VoiceType": "StandardVoice"
            }
        ]
    }-->
    <speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis"
        xmlns:mstts="http://www.w3.org/2001/mstts" xmlns:emo="http://www.w3.org/2009/10/emotionml"
        xml:lang="gu-IN">
        <voice name="gu-IN-DhwaniNeural">
            """
        + text
        + """
        </voice>
    </speak>
    """
    )

    speech_synthesis_result = speech_synthesizer.speak_ssml_async(ssml=ssml).get()

    if (
        speech_synthesis_result.reason
        == speechsdk.ResultReason.SynthesizingAudioCompleted
    ):
        pass
        # print(f"Audio transcription was successful!")
        # with wave.open(output_wave_file, "wb") as out_f:
        #     out_f.setnchannels(1)
        #     out_f.setsampwidth(2) # number of bytes
        #     out_f.setframerate(44100)
        #     out_f.writeframesraw(speech_synthesis_result.audio_data)
    elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_synthesis_result.cancellation_details
        print("Speech synthesis canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            if cancellation_details.error_details:
                print("Error details: {}".format(cancellation_details.error_details))
                print("Did you set the speech resource key and region values?")


 ## Sync `data` from `blob` to `local` folder

In [7]:
# if SYNC_BLOB_TO_LOCAL_FORCE_DOWNLOAD:
#     # Function reference: https://learn.microsoft.com/en-us/python/api/azure-storage-blob/azure.storage.blob.blobserviceclient?view=azure-python#azure-storage-blob-blobserviceclient-get-container-client
#     container_client = blob_service_client.get_container_client(
#         container=CONTAINER_NAME
#     )

#     for blob_name in tqdm(
#         iterable=container_client.list_blob_names(name_starts_with="data")
#     ):
#         # Function reference: https://learn.microsoft.com/en-us/python/api/azure-storage-blob/azure.storage.blob.blobserviceclient?view=azure-python#azure-storage-blob-blobserviceclient-get-blob-client
#         blob_client = blob_service_client.get_blob_client(
#             container=CONTAINER_NAME, blob=blob_name
#         )
#         blob_name_dirname = os.path.dirname(blob_name)
#         os.makedirs(name=blob_name_dirname, exist_ok=True)
#         if not os.path.exists(blob_name):
#             with open(blob_name, "wb") as local_file:
#                 download_stream = blob_client.download_blob()
#                 local_file.write(download_stream.readall())


## Content Creation Pipeline

### Steps

1. Get a Gujarati Kahevat in Gujarati with English Meaning
2. Get a Background Image from ...
3. Get a Background Music from ...

#### 1. Gujarati Kahevat

In [13]:
openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(
  model="text-davinci-003",
  prompt="Popular Gujarati Kahevat in Gujarati with Meaning in English",
  temperature=0.7,
  max_tokens=999,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [14]:
print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\n1. \u0aa8\u0ab0 \u0aa8\u0ab0\u0abe\u0a9c\u0aa8\u0ac1\u0a82 \u0aac\u0ab9\u0abe\u0ab0 \u0a85\u0aa8\u0ac7 \u0aa8\u0abe\u0ab0\u0abe\u0aaf\u0aa3\u0aa8\u0ac1\u0a82 \u0aa8\u0a9c\u0ab0 \u0aa4\u0ab0\u0aab \u0aa8\u0ab9\u0ac0:\n\nTranslation: No one is greater than the King, and no one should look down upon the poor."
    }
  ],
  "created": 1674227599,
  "id": "cmpl-6anMtx2e9D9nA2tZIn5WWUv1ZiUoP",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 137,
    "prompt_tokens": 15,
    "total_tokens": 152
  }
}


In [15]:
print(response['choices'][0]['text'])



1. નર નરાજનું બહાર અને નારાયણનું નજર તરફ નહી:

Translation: No one is greater than the King, and no one should look down upon the poor.


In [25]:
result_split = response['choices'][0]['text'].split('\n')
guj_kvt = result_split[2].split('1.')[1].strip()
eng_mean = result_split[4].strip()
print(guj_kvt, '\n', eng_mean)

નર નરાજનું બહાર અને નારાયણનું નજર તરફ નહી: 
 Translation: No one is greater than the King, and no one should look down upon the poor.


#### 2. Image from Bing

In [ ]:
content_creation_pipeline_folder = "data/" # add number folder

search_url = f"{os.environ['bing_endpoint']}v7.0/images/search"
subscription_key = os.environ['bing_subscription_key']
            
headers = {
    "Ocp-Apim-Subscription-Key" : subscription_key,
}
params = {
    "q": "inspiration, motivation", # Can also search for specific images
    "license": "all",
    "imageType": "photo",
    "modules": "Collections, BRQ, SimilarProducts, SimilarImages, RecognizedEntities, RelatedSearches", # Bing API insight token: https://learn.microsoft.com/en-us/bing/search-apis/bing-image-search/reference/query-parameters#insightstoken
    "cc": "IN", # Bing API Country Codes https://learn.microsoft.com/en-us/bing/search-apis/bing-image-search/reference/market-codes#country-codes
    #"setLang": "gu", # Bing API language codes https://learn.microsoft.com/en-us/bing/search-apis/bing-image-search/reference/market-codes#bing-supported-language-codes
}

response = requests.get(search_url, headers=headers, params=params)
response.raise_for_status()
search_results = response.json()

image_num = 0
nextOffset = search_results["nextOffset"]
os.makedirs(name=content_creation_pipeline_folder, exist_ok=True)

while image_num == 0:
    search_results["value"].sort(key=lambda element: element['datePublished'], reverse=True)

    for i in range(len(search_results)):
        if search_results["value"][i]["isFamilyFriendly"] == True:
            if "youtube" not in search_results["value"][i]["hostPageUrl"]:
                if image_num == 0:
                    image_num += 1

                    image_data = requests.get(search_results["value"][i]["thumbnailUrl"])
                    image_data.raise_for_status()
                    image = Image.open(BytesIO(image_data.content))
                    image.save(f"{content_creation_pipeline_folder}/image.jpg")

                else:
                    break

#### 3. Music

In [ ]:
content_creation_pipeline_folder = "data/" # add number folder

# Generate audio file describing the word whilst spoken in Gujarati by Azure Speech API (saved as wav file)
wave_file_output = f"{content_creation_pipeline_folder}/gujaratiDefinition.wav"
if not os.path.exists(wave_file_output):
    generate_transcription(
        text=guj_kvt,
        output_wave_file=wave_file_output,
    )

In [1]:
## Look at dynamic music synthesis models (e.g. Google Magenta) to produce some relaxing sounds

import magenta
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

# Load the MusicVAE model
config = configs.CONFIG_MAP['cat-mel_2bar_big']
model = TrainedModel(config)

# Generate a new MIDI file
generated_sequence = model.generate(length=16, temperature=1.0)
magenta.music.sequence_proto_to_midi_file(generated_sequence, 'generated.mid')

ModuleNotFoundError: No module named 'magenta'

## Prepare Content List

### Steps

1. Put the Text on Image
2. Create a animation of text on Image
3. Put Music in background
4. Save the video as a mp4

#### 1. Put Text on Image

In [ ]:
from PIL import ImageFont, ImageDraw

content_creation_pipeline_folder = "data/" # add number folder

bg_image = Image.open(f"{content_creation_pipeline_folder}/image.jpg")

txt = guj_kvt
txt_font = ImageFont.truetype('playfair/playfair-font.ttf', 200)    # https://fonts.google.com/
txt_pos = (15,15)
txt_color = (237, 230, 211)    # https://colorpicker.me/#84f760

image_editable = ImageDraw.Draw(bg_image)
image_editable.text(xy=txt_pos, text=txt, fill=txt_color, font=txt_font)

bg_image.save(f"{content_creation_pipeline_folder}/result.jpg")

#### 2. Animation

#### 3. Background Music Addition

#### 4. Video

In [ ]:
from moviepy.editor import *

# Load the image
image = ImageClip("image.jpg")

# Load the background music
music = AudioFileClip("bg_music.mp3")

# Create the final video by concatenating the image and music
final_video = concatenate_videoclips([image, music])

# Save the final video
final_video.write_videofile("video_with_image_and_music.mp4")

## Process Content List

### Steps

1. Upload to the blob
2. Upload to social network
